<a href="https://colab.research.google.com/github/ahcamachod/1819-clustering-extrayendo-patrones-de-datos/blob/main/clustering_aula_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ciencia de Datos - Clustering: Extrayendo patrones de datos
El objetivo de este proyecto es analizar patrones en el comportamiento de clientes  utilizando la técnica de  aprendizaje no supervisada llamada <b>Clustering</b> para identificar comportamiento y características de los grupos más representativos, y en base a las conclusiones poder diseñar estrategias para aumentar ingresos o disminuir pérdidas.Esto permitirá evaluar la efectividad de la estrategia en determinado grupo de clientes (cluster) , al tener como referencia clientes con patrones de comportamiento semejantes.

En este estudio no se pretende implementar ningun modelo de machine learnig pero si hacer uso de algunas técnicas del preprocesamiento de datos.

En este notebook empleamos un dataset de Kaggle llamado CC GENERAL.csv.

Dataset: https://www.kaggle.com/arjunbhasin2013/ccdata

### **Del proyecto:** 231103-1_data_science_clustering_extrayendo_patrones_datos

### **Procedimientos:**
1. Preparar el entorno de desarrollo en el cuaderno de Jupiter para Google Colab
2. Importar bibliotecas de los métodos a utilizar
3. Cargar el dataset en un DataFrame
4. Consultar información sobre las variables del Dataframe
5. Preprocesamiento:
    - Eliminar variables innecesarias o que no permitan leer el dataframe
    - Tratamiento de variables con valores faltantes
    - Tratamiento de datos extremos y datos atípicos
    - Normalizar los valores de todas las variables
6. Clusters: Criterios y métricas de validación
    - Coeficiente de Silhouette
    - Indice Davies-Bouldin
    - Indice Calinski-Harabasz
7. Validando y visualizando los clusters.
   - Validando la estructura de los closters
   - Validando la estabilidad de los closters
8. Entendiendo los valores de los atributos en los clusters
9. Interpretando los clusters.
10. Utilizando los clusters

### **1. Preparar el entorno de desarrollo en el cuaderno de Jupiter para Google Colab**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -r "/content/drive/MyDrive/Colab Notebooks/231103-1_machine_learning_clustering_extrayendo_patrones_datos/requirements.txt"

In [3]:
# How can I suppresed Python warnings in Jupyter Notebook
# Why not just set n_init=10 explicitly and retain the same value as the default and get rid of those warnings?
import warnings
warnings.filterwarnings('ignore')

### **2. Importar bibliotecas de los métodos a utilizar**





In [4]:
# Las versiones de las bibliotecas que usaremos son las siguientes:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn

print(f'matplotlib --> {matplotlib.__version__}')
print(f'pandas --> {pd.__version__}')
print(f'numpy --> {np.__version__}')
print(f'seaborn --> {sns.__version__}')
print(f'sklearn --> {sklearn.__version__}')

matplotlib --> 3.7.1
pandas --> 1.5.3
numpy --> 1.23.5
seaborn --> 0.12.2
sklearn --> 1.2.2


### **3. Cargar el dataset en un dataframe**

In [5]:
# ¿Cual es la manera recomendada para fijar la ruta de un archivo dataset para cargarlo en Google Colab?
# Respuesta: Asignar la ruta a un string y pasarlo como parametro al método pd.read_csv()
cc_data_file_path = '/content/drive/MyDrive/Colab Notebooks/231103-1_machine_learning_clustering_extrayendo_patrones_datos/datasets/CC_GENERAL.csv'
credit_card_df = pd.read_csv(cc_data_file_path)

In [ ]:
import pandas as pd

uri = "https://gist.githubusercontent.com/jessicadesousa/56ac5efd1b9f2a06ef503485e904a203/raw/d829a7204ad55506644f4a7058090c84424217e7/machine-learning-carros-simulacion.csv"
datos = pd.read_csv(uri)
datos.head()

### **4. Consultar información sobre las variables del Dataframe**

In [6]:
credit_card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 11  CASH_ADVANCE_TRX                  8950 non-null   int64  
 12  PURCHA

In [7]:
credit_card_df.head(3)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.0,0.083333,0.00,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.0,0.000000,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.0,0.000000,0.00,0,12,7500.0,622.066742,627.284787,0.000000,12


### Paso 1

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = datos[["precio", "edad_del_modelo", "km_por_ano"]]
y = datos["vendido"]

SEED = 158020
np.random.seed(SEED)

train_x, test_x, train_y, test_y = train_test_split(x, y, 
                                                    test_size = 0.25, 
                                                    stratify = y)

print("Entrenaremos con %d elementos y evaluaremos con %d elementos" %(len(train_x), len(test_y)))

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy="stratified")
dummy_stratified.fit(train_x, train_y)
accuracy = dummy_stratified.score(test_x, test_y) * 100

print("La exactitud(accuracy) del dummy stratified fue %.2f%%" % accuracy)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(train_x, train_y)

predicciones = modelo.predict(test_x)

accuracy = accuracy_score(test_y, predicciones) * 100
print("La exactitud(accuracy) fue %.2f%%" % accuracy)

In [ ]:
SEED = 5
np.random.seed(SEED)

train_x, test_x, train_y, test_y = train_test_split(x, y, 
                                                    test_size = 0.25, 
                                                    stratify = y)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(train_x, train_y)

predicciones = modelo.predict(test_x)

accuracy = accuracy_score(test_y, predicciones) * 100
print("La exactitud(accuracy) fue %.2f%%" % accuracy)